In [1]:
from utils import  select_and_save_tiles

In [1]:
name_file = "/home/mgallet/Documents/Herbiland/vegetation/DATA/LIDAR/RAW/peclerey/LHD_FXX_1007_6552_PTS_C_LAMB93_IGN69.copc.laz"
tuiles_path = "/home/mgallet/Documents/DATA/TA_programme_LiDAR-HD/TA_programme_lidar_HD_path.shp"
parcelle_path = "/home/mgallet/Documents/Herbiland/vegetation/DATA/SHP/Bauges_Belledonne_MB/Emprise_totale_3massifs_EPSG2154_V2.shp"

# o = select_and_save_tiles(tuiles_path, parcelle_path, name_file, "test.shp")

In [4]:
import pdal, json

pdal_pipeline = json.dumps(
            {
  "pipeline": [
    {
      "type": "readers.las",
      "filename": "/home/mgallet/Documents/Herbiland/vegetation/DATA/LIDAR/RAW/peclerey/LHD_FXX_1007_6552_PTS_C_LAMB93_IGN69.copc.laz"
    },
    
  ]
    }
)
pipeline = pdal.Pipeline(pdal_pipeline)
pipeline.execute()


17322128

In [7]:
pipeline.arrays[0]
import os 
os.sys.getsizeof(pipeline.arrays[0])/1024/1024

842.5032043457031

In [ ]:
o.descr

TypeError: list indices must be integers or slices, not str

In [ ]:
a


array([['X', '<f8'],
       ['Y', '<f8'],
       ['Z', '<f8'],
       ['Intensity', '<u2'],
       ['ReturnNumber', '|u1'],
       ['NumberOfReturns', '|u1'],
       ['ScanDirectionFlag', '|u1'],
       ['EdgeOfFlightLine', '|u1'],
       ['Classification', '|u1'],
       ['Synthetic', '|u1'],
       ['KeyPoint', '|u1'],
       ['Withheld', '|u1'],
       ['Overlap', '|u1'],
       ['ScanAngleRank', '<f4'],
       ['UserData', '|u1'],
       ['PointSourceId', '<u2'],
       ['GpsTime', '<f8'],
       ['ScanChannel', '|u1']], dtype='<U17')

In [ ]:
"X" in np.array(o.descr)


var_for

[('X', '<f8'),
 ('Y', '<f8'),
 ('Z', '<f8'),
 ('Intensity', '<u2'),
 ('Classification', '|u1')]

In [74]:
for i in sub.dtype.names:
    print(i)

X
Y
Z
Intensity


In [ ]:
import numpy as np
keep_variables= [
    "X",
    "Y",
    "Z",
    "Intensity",
    "Classification",
    "HeightAboveGround"
]
o =pipeline.arrays[0].dtype
o.descr[0][1]
sub = np.empty(pipeline.arrays[0].shape[0], dtype=[('X', 'f8'), ('Y', 'f8'), ('Z', 'f8'), ('Intensity', 'u2')])
sub['X'] = pipeline.arrays[0]['X']
sub['Y'] = pipeline.arrays[0]['Y']
sub['Z'] = pipeline.arrays[0]['Z']
sub['Intensity'] = pipeline.arrays[0]['Intensity']

np.save("test.npy", sub)
del sub
sub = np.load("test.npy")
os.sys.getsizeof(sub)/1024/1024



429.51148986816406

In [76]:
import json

with open("config/default_config.json", "r") as f:
    config = json.load(f)

config["processor_settings"]

{'group': 5,
 'keep_variables': ['X',
  'Y',
  'Z',
  'Intensity',
  'Classification',
  'HeightAboveGround'],
 'n_jobs': 2}